In [1]:
import numpy as np
from ompl import base as ob
from ompl import geometric as og
from pydrake.planning import RobotDiagramBuilder
from pydrake.math import RigidTransform
from airo_drake import add_meshcat, finish_build, visualize_frame
import airo_models
from dataclasses import dataclass
from ConstrainedPlanningCommon import ConstrainedProblem

In [ ]:
# Create the robot diagram builder
robot_diagram_builder = RobotDiagramBuilder()

# Add meshcat visualizer
meshcat = add_meshcat(robot_diagram_builder)

# Get plant and parser from the builder
plant = robot_diagram_builder.plant()
parser = robot_diagram_builder.parser()
parser.SetAutoRenaming(True)

# Create two cubes using airo_models
cube1_size = (0.2, 0.2, 0.2)
cube2_size = (0.2, 0.2, 0.2)
cube1_urdf_path = airo_models.box_urdf_path(cube1_size, "cube1")
cube2_urdf_path = airo_models.box_urdf_path(cube2_size, "cube2")

# Add cubes to the scene
cube1_index = parser.AddModels(cube1_urdf_path)[0]
cube2_index = parser.AddModels(cube2_urdf_path)[0]

# Position the cubes in space
world_frame = plant.world_frame()
cube1_frame = plant.GetFrameByName("base_link", cube1_index)
cube2_frame = plant.GetFrameByName("base_link", cube2_index)

# Set transforms for the cubes
cube1_transform = RigidTransform(p=[0.5, 0.0, 0.3])  # Position cube1 at (0.5, 0, 0.3)
cube2_transform = RigidTransform(p=[-0.5, 0.0, 0.3])  # Position cube2 at (-0.5, 0, 0.6)

# Weld the cubes to the world frame at their positions
plant.WeldFrames(world_frame, cube1_frame, cube1_transform)
plant.WeldFrames(world_frame, cube2_frame, cube2_transform)

# Finish building the scene
robot_diagram, context = finish_build(robot_diagram_builder, meshcat)

# Force publish to visualize
robot_diagram.ForcedPublish(context)

# Visualize the frames of the cubes
visualize_frame(meshcat, "cube1_frame", cube1_transform, length=0.25)
visualize_frame(meshcat, "cube2_frame", cube2_transform, length=0.25)

In [3]:
# Define goal state
goal_transform1 = RigidTransform(p=[0.3, 0.4, 0.5])
goal_transform2 = RigidTransform(p=[-0.3, 0.4, 0.5])

In [4]:
class TwoCubeConstraint(ob.Constraint):
    def __init__(self, fixed_distance=0.6, sphere_radius=1.0):
        # 6 dimensions (3D position for each cube)
        # 3 constraints: fixed distance + 2 sphere constraints
        super(TwoCubeConstraint, self).__init__(6, 3)
        self.fixed_distance = fixed_distance
        self.sphere_radius = sphere_radius
        
    def function(self, x, out):
        # Extract positions of both cubes
        p1 = x[:3]
        p2 = x[3:]
        
        # Constraint 1: Fixed distance between cubes
        dist = np.linalg.norm(p2 - p1)
        out[0] = dist - self.fixed_distance
        
        # Constraint 2 & 3: Both cubes must stay within sphere
        out[1] = np.linalg.norm(p1) - self.sphere_radius
        out[2] = np.linalg.norm(p2) - self.sphere_radius
        
        return True
        
    def jacobian(self, x, out):
        p1 = x[:3]
        p2 = x[3:]
        
        # Initialize Jacobian matrix
        out[:,:] = np.zeros((self.getCoDimension(), self.getAmbientDimension()))
        
        # Jacobian for distance constraint
        diff = p2 - p1
        norm = np.linalg.norm(diff)
        if norm > 0:
            grad = diff / norm
            out[0, :3] = -grad
            out[0, 3:] = grad
            
        # Jacobian for sphere constraints
        if np.linalg.norm(p1) > 0:
            out[1, :3] = p1 / np.linalg.norm(p1)
        if np.linalg.norm(p2) > 0:
            out[2, 3:] = p2 / np.linalg.norm(p2)
            
        return True

In [ ]:
@dataclass
class Options:
    space: str = "PJ"  # Projection-based state space
    planner: str = "RRTConnect"
    delta: float = ob.CONSTRAINED_STATE_SPACE_DELTA
    lambda_: float = ob.CONSTRAINED_STATE_SPACE_LAMBDA
    tolerance: float = ob.CONSTRAINT_PROJECTION_TOLERANCE
    time: float = 50.0
    tries: int = ob.CONSTRAINT_PROJECTION_MAX_ITERATIONS
    range: float = 0.0

# Setup OMPL planning
space = ob.RealVectorStateSpace(6)  # 6D state space (3D + 3D)

# Set bounds for the space
bounds = ob.RealVectorBounds(6)
for i in range(6):
    bounds.setLow(i, -1.5)  # Slightly larger than sphere radius
    bounds.setHigh(i, 1.5)
space.setBounds(bounds)

# Create constraint and problem
constraint = TwoCubeConstraint()
options = Options()
problem = ConstrainedProblem(options.space, space, constraint, options)

# Set start and goal states
start = ob.State(problem.css)
goal = ob.State(problem.css)

In [ ]:
# Set start state (current position of cubes)
start()[0] = cube1_transform.translation()[0]
start()[1] = cube1_transform.translation()[1]
start()[2] = cube1_transform.translation()[2]
start()[3] = cube2_transform.translation()[0]
start()[4] = cube2_transform.translation()[1]
start()[5] = cube2_transform.translation()[2]

# Set goal state
goal()[0] = goal_transform.translation()[0]
goal()[1] = goal_transform.translation()[1]
goal()[2] = goal_transform.translation()[2]
# # The second cube's goal position is not strictly specified
# goal()[3] = -0.5  # Example position that maintains constraints
# goal()[4] = 0.0
# goal()[5] = 0.3

problem.setStartAndGoalStates(start, goal)
problem.setPlanner(options.planner)

# Solve the problem
solved = problem.solveOnce(output=True)

if solved:
    # Get the solution path
    path = problem.ss.getSolutionPath()
    path.interpolate()
    
    # Visualize the solution
    for i in range(path.getStateCount()):
        state = path.getState(i)
        
        # Create transforms for visualization
        cube1_transform_i = RigidTransform(p=[state[0], state[1], state[2]])
        cube2_transform_i = RigidTransform(p=[state[3], state[4], state[5]])
        
        # Visualize frames
        visualize_frame(meshcat, f"cube1_frame_{i}", cube1_transform_i, length=0.15)
        visualize_frame(meshcat, f"cube2_frame_{i}", cube2_transform_i, length=0.15)
        
        # Optional: add small delay to see motion
        import time
        time.sleep(0.1)
else:
    print("No solution found")